# Crear  Tensores

In [ ]:

import torch
import json
from collections import defaultdict

# Cargar los archivos JSON de entrada y salida
with open("../data/processed/model/input_data.json", "r") as f:
    input_data_json = json.load(f)

with open("../data/processed/model/reference_data.json", "r") as f:
    output_data_json = json.load(f)

# Agrupar los datos por scenario_id
input_by_scenario = defaultdict(list)
output_by_scenario = defaultdict(list)

# Agrupar datos de entrada por scenario_id
for entry in input_data_json:
    scenario_id = entry["scenario_id"]
    input_by_scenario[scenario_id].append(entry)

# Agrupar datos de salida por scenario_id
for entry in output_data_json:
    scenario_id = entry["scenario_id"]
    output_by_scenario[scenario_id].append(entry)

# Obtener la lista de escenarios (deberían ser 68)
scenarios = sorted(input_by_scenario.keys())
if len(scenarios) != 68:
    print(f"Advertencia: Se esperaban 68 escenarios, pero se encontraron {len(scenarios)}")

# Crear listas para almacenar los datos de entrada y salida
input_data = []
target_data = []

# Función para convertir valores a float de manera segura
def safe_float(value, default=0.0):
    try:
        return float(value)
    except (ValueError, TypeError):
        return default

# Procesar cada escenario
for scenario_id in scenarios:
    scenario_inputs_list = input_by_scenario[scenario_id]
    scenario_outputs_list = output_by_scenario[scenario_id]

    # Verificar que el número de pares coincida entre entrada y salida
    if len(scenario_inputs_list) != len(scenario_outputs_list):
        print(f"Advertencia: El escenario {scenario_id} tiene {len(scenario_inputs_list)} entradas y {len(scenario_outputs_list)} salidas")

    scenario_inputs = []
    scenario_targets = []

    # Procesar cada par de relés en el escenario (hasta 100)
    num_pairs = min(len(scenario_inputs_list), 100)
    for pair_idx in range(num_pairs):
        pair_input = scenario_inputs_list[pair_idx]
        pair_output = scenario_outputs_list[pair_idx] if pair_idx < len(scenario_outputs_list) else None

        # Datos de entrada
        inputs = [
            # safe_float(pair_input["fault"]),
            safe_float(pair_input["main_relay"]["Ishc"]),
            safe_float(pair_input["main_relay"]["Time_out"]),
            safe_float(pair_input["main_relay"]["current_a"]),
            safe_float(pair_input["backup_relay"]["Ishc"]),
            safe_float(pair_input["backup_relay"]["Time_out"]),
            safe_float(pair_input["backup_relay"]["current_a"])
        ]
        scenario_inputs.append(inputs)

        # Datos de salida (si existen)
        if pair_output:
            targets = [
                safe_float(pair_output["main_relay"]["pick_up"]),
                safe_float(pair_output["main_relay"]["TDS"]),
                safe_float(pair_output["backup_relay"]["pick_up"]),
                safe_float(pair_output["backup_relay"]["TDS"])
            ]
        else:
            targets = [0.0] * 4  # Rellenar con ceros si no hay salida correspondiente
        scenario_targets.append(targets)

    # Rellenar con ceros si hay menos de 100 pares
    while len(scenario_inputs) < 100:
        scenario_inputs.append([0.0] * 6)
        scenario_targets.append([0.0] * 4)

    input_data.append(scenario_inputs)
    target_data.append(scenario_targets)

# Convertir las listas a tensores de PyTorch
input_tensor = torch.tensor(input_data, dtype=torch.float32)  # Forma: (68, 100, 6)
target_tensor = torch.tensor(target_data, dtype=torch.float32)  # Forma: (68, 100, 4)

# Verificar las formas de los tensores
print("Forma del tensor de entrada:", input_tensor.shape)
print("Forma del tensor de salida:", target_tensor.shape)



# Implementación del transformer 

In [20]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # Importado para F.relu
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import optuna
import math
import time # Para medir tiempo
import traceback # Para imprimir errores detallados

# --- 1. Carga y Preprocesamiento de Datos ---
# ######################################################################### #
# ############# ¡¡¡IMPORTANTE!!! ############# #
# REEMPLAZA ESTA SECCIÓN CON TU CÓDIGO REAL PARA CARGAR Y PREPARAR #
# 'input_tensor' y 'target_tensor' #
# Deben tener la forma: (num_escenarios, secuencia_len, num_features) #
# Y ser tensores de PyTorch. #
# ######################################################################### #
print("Cargando y preprocesando datos...")
# Ejemplo Placeholder (DEBES REEMPLAZAR ESTO):
try:
    # Intenta cargar desde archivos si ya los tienes guardados
    input_tensor = torch.load('input_tensor.pt')
    target_tensor = torch.load('target_tensor.pt')
    print("Tensores cargados desde archivos .pt")
except FileNotFoundError:
    print("Archivos .pt no encontrados, generando datos de ejemplo...")
    # Generar datos de ejemplo si no se encuentran archivos
    num_escenarios = 500 # Reducido para ejemplo rápido
    secuencia_len = 74   # Número de pares de relés
    input_features = 6
    output_features = 4
    input_tensor = torch.rand(num_escenarios, secuencia_len, input_features, dtype=torch.float32)
    # Asegurar que los datos objetivo de ejemplo sean no negativos
    target_tensor = torch.abs(torch.randn(num_escenarios, secuencia_len, output_features, dtype=torch.float32)) * 5
    # Guardar para la próxima vez (opcional)
    # torch.save(input_tensor, 'input_tensor.pt')
    # torch.save(target_tensor, 'target_tensor.pt')
    print("Datos de ejemplo generados.")
except Exception as e:
     print(f"Error al cargar/generar datos: {e}. Saliendo.")
     exit()

# Asegurar que los tensores son float32
input_tensor = input_tensor.float()
target_tensor = target_tensor.float()

print(f"Forma Input Tensor Original: {input_tensor.shape}")
print(f"Forma Target Tensor Original: {target_tensor.shape}")
# ######################################################################### #
# ############# FIN DE LA SECCIÓN A REEMPLAZAR ############# #
# ######################################################################### #


# --- 2. Normalización ---
print("Normalizando datos...")
# Guardar scalers para uso posterior
scaler_input = StandardScaler()
scaler_target = StandardScaler()

try:
    # Aplanar para ajustar los scalers
    input_np_original = input_tensor.numpy().reshape(-1, input_tensor.shape[-1])
    target_np_original = target_tensor.numpy().reshape(-1, target_tensor.shape[-1])

    # Validar que no haya NaNs antes de escalar
    if np.isnan(input_np_original).any() or np.isinf(input_np_original).any():
        raise ValueError("Datos de entrada originales contienen NaNs o Infinitos.")
    if np.isnan(target_np_original).any() or np.isinf(target_np_original).any():
         raise ValueError("Datos objetivo originales contienen NaNs o Infinitos.")

    # Ajustar y transformar
    input_normalized = scaler_input.fit_transform(input_np_original)
    target_normalized = scaler_target.fit_transform(target_np_original)

    # Validar que no haya NaNs después de escalar
    if np.isnan(input_normalized).any() or np.isinf(input_normalized).any():
        raise ValueError("Datos de entrada contienen NaNs/Inf DESPUÉS de normalizar.")
    if np.isnan(target_normalized).any() or np.isinf(target_normalized).any():
        raise ValueError("Datos objetivo contienen NaNs/Inf DESPUÉS de normalizar.")

    # Restaurar forma original
    input_tensor_normalized = torch.tensor(input_normalized, dtype=torch.float32).reshape(input_tensor.shape)
    target_tensor_normalized = torch.tensor(target_normalized, dtype=torch.float32).reshape(target_tensor.shape)
    print("Datos normalizados correctamente.")

except Exception as e:
    print(f"Error durante la normalización: {e}")
    traceback.print_exc()
    exit()


# --- 3. División Entrenamiento / Validación e Identificación ---
print("Dividiendo datos...")
num_total_escenarios = input_tensor_normalized.shape[0]
indices = list(range(num_total_escenarios)) # Usar lista para train_test_split
test_split_percentage = 0.2
random_seed = 42

try:
    train_idx, val_idx = train_test_split(indices, test_size=test_split_percentage, random_state=random_seed, shuffle=True)
except ValueError as e:
     print(f"Error en train_test_split: {e}. ¿Hay suficientes datos?")
     exit()

# Convertir a listas y ordenar para facilitar visualización/manejo
train_idx = sorted(list(train_idx))
val_idx = sorted(list(val_idx))

# --- Impresión de detalles de la división ---
print("\n" + "="*60)
print("--- DETALLES DE LA DIVISIÓN DE DATOS ---")
print(f"Número total de escenarios originales: {num_total_escenarios}")
print(f"Porcentaje para validación (test_size): {test_split_percentage * 100:.1f}%")
print(f"Semilla aleatoria (random_state): {random_seed}")
print("-" * 60)
print(f"Número de escenarios para ENTRENAMIENTO: {len(train_idx)}")
print(f"Índices originales usados para ENTRENAMIENTO (primeros 50):")
print(train_idx[:50])
if len(train_idx) > 50: print("...")
print("-" * 60)
print(f"Número de escenarios para VALIDACIÓN: {len(val_idx)}")
print(f"Índices originales usados para VALIDACIÓN (primeros 50):")
print(val_idx[:50])
if len(val_idx) > 50: print("...")
print("=" * 60 + "\n")
# --- Fin Impresión ---

# Crear los tensores de entrenamiento/validación usando los índices
try:
    train_input = input_tensor_normalized[train_idx]
    train_target = target_tensor_normalized[train_idx]
    val_input = input_tensor_normalized[val_idx]
    val_target = target_tensor_normalized[val_idx]
    print(f"Forma datos entrenamiento (Input): {train_input.shape}")
    print(f"Forma datos validación (Input): {val_input.shape}")
except IndexError:
     print("Error al crear subconjuntos de datos. ¿Coinciden los índices con las dimensiones?")
     exit()

# --- 4. Definición del Modelo Transformer (con ReLU al final) ---

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model) # Shape: (1, max_len, d_model) para batch_first
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """ Args: x: Tensor, shape [batch_size, seq_len, d_model] """
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, d_model, nhead, num_encoder_layers, dim_feedforward, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.d_model = d_model

        self.input_proj = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward,
            dropout=dropout, batch_first=True, activation=F.gelu
        )
        encoder_norm = nn.LayerNorm(d_model)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers, norm=encoder_norm)

        self.output_proj = nn.Linear(d_model, output_dim)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.input_proj.weight, -initrange, initrange)
        nn.init.zeros_(self.input_proj.bias)
        nn.init.uniform_(self.output_proj.weight, -initrange, initrange)
        nn.init.zeros_(self.output_proj.bias)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        src = self.input_proj(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        output = self.output_proj(output)
        output = F.relu(output) # Asegurar salida no negativa
        return output

# --- 5. Funciones de Entrenamiento, Evaluación y Predicción ---

def train_epoch(model, data_loader, criterion, optimizer, device, scaler=None):
    model.train()
    total_loss = 0.0
    for batch_input, batch_target in data_loader:
        batch_input, batch_target = batch_input.to(device), batch_target.to(device)
        optimizer.zero_grad(set_to_none=True)

        use_amp_here = scaler is not None
        with torch.cuda.amp.autocast(enabled=use_amp_here):
            output = model(batch_input)
            loss = criterion(output, batch_target)

        if use_amp_here:
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer) # Desescalar antes de clip
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

        total_loss += loss.item()
    return total_loss / len(data_loader)

def evaluate_epoch(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input, batch_target = batch_input.to(device), batch_target.to(device)
            with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')): # AMP también en eval
                 output = model(batch_input)
                 loss = criterion(output, batch_target)
            total_loss += loss.item()
    return total_loss / len(data_loader)

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, model_save_path='best_transformer_model.pth', use_amp=False):
    best_val_loss = float('inf')
    epochs_no_improve = 0
    patience = 15
    scaler = torch.cuda.amp.GradScaler() if use_amp and device.type == 'cuda' else None
    if scaler: print("Usando Precisión Mixta Automática (AMP) para el entrenamiento.")

    for epoch in range(num_epochs):
        start_time = time.time()
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device, scaler)
        val_loss = evaluate_epoch(model, val_loader, criterion, device)
        end_time = time.time()

        # Validar pérdida de validación
        if np.isnan(val_loss) or np.isinf(val_loss):
            print(f"Epoch {epoch+1:03d}/{num_epochs}: Pérdida de validación inválida ({val_loss}). Deteniendo entrenamiento.")
            break # Detener si la pérdida diverge

        lr_current = optimizer.param_groups[0]['lr']
        print(f'Epoch {epoch+1:03d}/{num_epochs}, Train Loss: {train_loss:.5f}, Val Loss: {val_loss:.5f}, LR: {lr_current:.6f}, Time: {end_time - start_time:.2f}s')

        if scheduler:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                 scheduler.step(val_loss)
            # Añadir 'else' si usas otros tipos de scheduler que requieran step()

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            try:
                torch.save(model.state_dict(), model_save_path)
                print(f'    --> Modelo guardado en {model_save_path} (Val Loss: {best_val_loss:.5f})')
            except Exception as e:
                print(f"    --> ERROR al guardar el modelo: {e}")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f'    --> Early stopping activado tras {patience} épocas sin mejora.')
            break

    print(f'Entrenamiento finalizado. Mejor Val Loss: {best_val_loss:.5f}')
    # Cargar el mejor modelo al final para devolverlo
    if best_val_loss != float('inf'): # Solo cargar si se guardó algo
        try:
            model.load_state_dict(torch.load(model_save_path))
            print(f"Mejor modelo cargado desde {model_save_path}")
        except Exception as e:
            print(f"Error al cargar el mejor modelo guardado: {e}")
    else:
        print("Advertencia: No se guardó ningún modelo (la pérdida de validación nunca mejoró).")

    return best_val_loss, model

# --- Función para Predicción de un Solo Escenario ---
def predict_single_scenario(model, scenario_input_original, scaler_input, scaler_target, device):
    model.eval()
    if isinstance(scenario_input_original, torch.Tensor):
        input_np_original = scenario_input_original.cpu().numpy()
    else:
        input_np_original = np.array(scenario_input_original)

    if input_np_original.ndim != 2:
         raise ValueError(f"La entrada del escenario debe ser 2D (seq_len, features), pero tiene forma {input_np_original.shape}")

    seq_len, expected_features = input_np_original.shape
    if not hasattr(scaler_input, 'n_features_in_') or expected_features != scaler_input.n_features_in_:
         # Añadir chequeo por si el scaler no está ajustado o las features no coinciden
         raise ValueError(f"Inconsistencia en características de entrada: Scaler espera {getattr(scaler_input, 'n_features_in_', 'N/A')} features, datos tienen {expected_features}.")

    input_np_normalized = scaler_input.transform(input_np_original)
    input_tensor_norm = torch.tensor(input_np_normalized, dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
             predicted_output_normalized = model(input_tensor_norm)

    pred_np_norm = predicted_output_normalized.squeeze(0).cpu().numpy()
    pred_np_denorm = scaler_target.inverse_transform(pred_np_norm)
    pred_np_denorm = np.maximum(pred_np_denorm, 0) # Clip final
    return pred_np_denorm


# --- 6. Optimización de Hiperparámetros con Optuna ---

# Obtener dimensiones y dispositivo
try:
    INPUT_DIM = train_input.shape[-1]
    OUTPUT_DIM = train_target.shape[-1]
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))
    print(f"Usando dispositivo: {DEVICE}")
    print(f"Input dim: {INPUT_DIM}, Output dim: {OUTPUT_DIM}")
    USE_AMP = (DEVICE.type == 'cuda') # Usar AMP solo si es CUDA
except Exception as e:
    print(f"Error al determinar dimensiones o dispositivo: {e}. Saliendo.")
    exit()

def objective(trial):
    d_model = trial.suggest_categorical('d_model', [64, 128, 256])
    ff_multiplier = trial.suggest_categorical('ff_multiplier', [2, 4])
    dim_feedforward = d_model * ff_multiplier
    trial.set_user_attr('calculated_dim_feedforward', dim_feedforward)

    possible_nheads = [h for h in [2, 4, 8, 16] if d_model % h == 0] # Añadido 16
    if not possible_nheads: nhead = 2
    else: nhead = trial.suggest_categorical('nhead', possible_nheads)

    num_encoder_layers = trial.suggest_int('num_encoder_layers', 1, 6) # Aumentado rango
    dropout = trial.suggest_float('dropout', 0.05, 0.35) # Rango ajustado
    learning_rate = trial.suggest_float('learning_rate', 5e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128]) # Añadido 128
    weight_decay = trial.suggest_float('weight_decay', 1e-7, 1e-3, log=True)

    # DataLoaders para el trial
    try:
        train_loader = DataLoader(TensorDataset(train_input, train_target), batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True, persistent_workers=True)
        val_loader = DataLoader(TensorDataset(val_input, val_target), batch_size=batch_size, num_workers=2, pin_memory=True, persistent_workers=True)
    except Exception: # Fallback más general
        print("Warning: DataLoader con workers/pin_memory falló, usando configuración básica.")
        train_loader = DataLoader(TensorDataset(train_input, train_target), batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(TensorDataset(val_input, val_target), batch_size=batch_size)

    model = TransformerModel(
        input_dim=INPUT_DIM, output_dim=OUTPUT_DIM, d_model=d_model, nhead=nhead,
        num_encoder_layers=num_encoder_layers, dim_feedforward=dim_feedforward, dropout=dropout
    ).to(DEVICE)

    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=False)
    scaler_optuna = torch.cuda.amp.GradScaler() if USE_AMP else None

    num_optuna_epochs = 30
    print(f"\n--- Optuna Trial {trial.number} ---")
    print(f"Params: d_model={d_model}, nhead={nhead}, layers={num_encoder_layers}, ff={dim_feedforward}({ff_multiplier}x), drop={dropout:.3f}, lr={learning_rate:.6f}, bs={batch_size}, wd={weight_decay:.7f}")

    best_trial_val_loss = float('inf')
    epochs_no_improve = 0
    patience_optuna = 7

    for epoch in range(num_optuna_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, DEVICE, scaler_optuna)
        val_loss = evaluate_epoch(model, val_loader, criterion, DEVICE)

        if np.isnan(val_loss) or np.isinf(val_loss):
             print(f"    --> Trial {trial.number} divergió (Val Loss: {val_loss}). Podando.")
             raise optuna.TrialPruned()

        print(f'    Epoch {epoch+1:02d}/{num_optuna_epochs}, Train Loss: {train_loss:.5f}, Val Loss: {val_loss:.5f}')
        if scheduler: scheduler.step(val_loss)

        if val_loss < best_trial_val_loss:
            best_trial_val_loss = val_loss
            epochs_no_improve = 0
        else:
             epochs_no_improve += 1

        if epochs_no_improve >= patience_optuna:
             print(f'    --> Optuna trial early stopping at epoch {epoch+1}. Val Loss: {best_trial_val_loss:.5f}')
             break

        trial.report(val_loss, epoch)
        if trial.should_prune():
            print(f"    --> Trial {trial.number} podado en la época {epoch+1}.")
            raise optuna.TrialPruned()

    return best_trial_val_loss

# Ejecutar Optuna
print("\n" + "="*60)
print("--- INICIANDO OPTIMIZACIÓN CON OPTUNA ---")
study_name = "transformer_optimization_v1" # Nombre para posible reanudación
storage_name = f"sqlite:///{study_name}.db" # Guardar estudio en archivo sqlite

try:
    # Intentar cargar estudio existente o crear uno nuevo
    study = optuna.create_study(
        study_name=study_name,
        storage=storage_name,
        direction='minimize',
        sampler=optuna.samplers.TPESampler(seed=random_seed),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), # Podar tras 5 épocas
        load_if_exists=True # Cargar si ya existe
    )
    print(f"Estudio Optuna '{study_name}' cargado/creado desde '{storage_name}'.")
    print(f"Número de trials ya completados: {len(study.trials)}")

    # Ejecutar optimize
    study.optimize(
        objective,
        n_trials=25, # Número de NUEVOS trials a ejecutar
        timeout=None, # Sin límite de tiempo
        n_jobs=1, # Usar 1 job para evitar problemas con GPU/MPS
        catch=(ValueError, RuntimeError, ) # Capturar errores comunes sin detener todo el estudio
    )

except KeyboardInterrupt:
    print("\nOptimización interrumpida por el usuario.")
except Exception as e:
    print(f"\nError durante la optimización de Optuna: {e}")
    traceback.print_exc()

# Obtener mejores parámetros
print("\n" + "="*60)
print("--- OPTIMIZACIÓN FINALIZADA ---")
best_params = {}
best_value = float('inf')

try:
    if study.best_trial:
        best_params = study.best_trial.params
        best_value = study.best_trial.value
        print(f"Mejor Valor (Val Loss): {best_value:.5f}")
        print("Mejores Hiperparámetros encontrados:")
        print(best_params)
        # Añadir valor calculado
        if 'ff_multiplier' in best_params and 'd_model' in best_params:
            best_params['calculated_dim_feedforward'] = best_params['d_model'] * best_params['ff_multiplier']
    else:
        print("No se encontró un 'best_trial' en el estudio Optuna.")

except Exception as e:
    print(f"Error al obtener resultados de Optuna: {e}")

# Fallback a parámetros por defecto si Optuna falló o no encontró nada
if not best_params:
    print("Usando parámetros por defecto debido a fallo o falta de resultados en Optuna.")
    best_params = {'d_model': 128, 'ff_multiplier': 4, 'nhead': 4, 'num_encoder_layers': 3,
                   'dropout': 0.15, 'learning_rate': 0.0005, 'batch_size': 64, 'weight_decay': 1e-5}
    # Calcular dim_feedforward para los por defecto
    best_params['calculated_dim_feedforward'] = best_params['d_model'] * best_params['ff_multiplier']
    print(f"Parámetros por defecto: {best_params}")


# Ajuste final de nhead (redundante si la lógica en objective es correcta)
if 'd_model' in best_params:
    d_model = best_params['d_model']
    possible_nheads = [h for h in [2, 4, 8, 16] if d_model % h == 0]
    if not possible_nheads:
        best_params['nhead'] = 2 # Fallback
    elif 'nhead' not in best_params or best_params['nhead'] not in possible_nheads:
        best_params['nhead'] = possible_nheads[0] # Elegir el primero compatible
        print(f"Ajuste post-optuna: nhead establecido a {best_params['nhead']} para ser compatible con d_model={d_model}")
else:
     # Si ni siquiera d_model está, algo falló gravemente
     print("Error crítico: 'd_model' no encontrado en best_params. Usando nhead=4 por defecto.")
     best_params['nhead'] = 4


# --- 7. Entrenamiento Final con los Mejores Hiperparámetros ---
print("\n" + "="*60)
print("--- INICIANDO ENTRENAMIENTO FINAL ---")
print(f"Usando parámetros: {best_params}")

# Crear DataLoaders finales
final_batch_size = best_params.get('batch_size', 64) # Usar get con default
try:
    final_train_loader = DataLoader(TensorDataset(train_input, train_target), batch_size=final_batch_size, shuffle=True, num_workers=2, pin_memory=True, persistent_workers=True)
    final_val_loader = DataLoader(TensorDataset(val_input, val_target), batch_size=final_batch_size, num_workers=2, pin_memory=True, persistent_workers=True)
except Exception:
    print("Warning: DataLoader con workers/pin_memory falló, usando configuración básica.")
    final_train_loader = DataLoader(TensorDataset(train_input, train_target), batch_size=final_batch_size, shuffle=True)
    final_val_loader = DataLoader(TensorDataset(val_input, val_target), batch_size=final_batch_size)

# Crear el modelo final
try:
    final_model = TransformerModel(
        input_dim=INPUT_DIM,
        output_dim=OUTPUT_DIM,
        d_model=best_params['d_model'],
        nhead=best_params['nhead'],
        num_encoder_layers=best_params['num_encoder_layers'],
        dim_feedforward=best_params['calculated_dim_feedforward'], # Usa el valor calculado
        dropout=best_params['dropout']
    ).to(DEVICE)
except KeyError as e:
     print(f"Error: Falta el hiperparámetro '{e}' en best_params. No se puede crear el modelo final.")
     exit()

criterion = nn.MSELoss()
optimizer = optim.AdamW(final_model.parameters(), lr=best_params['learning_rate'], weight_decay=best_params['weight_decay'])
final_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5, verbose=True)

# Entrenar
final_num_epochs = 150 # Ajusta según necesidad
model_save_filename = 'best_transformer_model_relu.pth'
best_final_loss, final_model = train_model(
    final_model, final_train_loader, final_val_loader, criterion, optimizer, final_scheduler,
    num_epochs=final_num_epochs, device=DEVICE, model_save_path=model_save_filename, use_amp=USE_AMP
)

print("\n" + "="*60)
print("--- SCRIPT DE ENTRENAMIENTO FINALIZADO ---")
print(f"Modelo final (con ReLU) entrenado. Mejor Loss de Validación: {best_final_loss:.5f}")
print(f"Modelo guardado como '{model_save_filename}'.")
print("Los scalers (scaler_input, scaler_target) son necesarios para usar el modelo en datos nuevos.")
print("Los índices de entrenamiento/validación están en 'train_idx' y 'val_idx'.")


# --- 8. Prueba con un Escenario Específico (Usando la función definida) ---
print("\n" + "="*60)
print("--- PRUEBA CON UN ESCENARIO ESPECÍFICO ---")

# Asegurarse de que las variables necesarias existen
required_vars_test = ['final_model', 'scaler_input', 'scaler_target', 'input_tensor', 'target_tensor', 'DEVICE']
if all(var in globals() for var in required_vars_test):

    # === PASO 1: Elige el índice del escenario a probar ===
    if 'val_idx' in globals() and len(val_idx) > 0:
        test_scenario_index = val_idx[0] # Primer escenario de validación
        print(f"Se probará con el escenario de índice original: {test_scenario_index} (pertenece al conjunto de validación)")
    else:
        test_scenario_index = 0 # Primer escenario del dataset
        if test_scenario_index < input_tensor.shape[0]:
             print(f"Se probará con el escenario de índice original: {test_scenario_index} (No hay datos de validación, usando índice 0)")
        else:
             print(f"Error: No hay datos disponibles para probar (índice {test_scenario_index} inválido).")
             test_scenario_index = -1 # Marcar como inválido

    if test_scenario_index != -1:
        # === PASO 2: Obtén los datos ORIGINALES (antes de normalizar) ===
        try:
            single_input_original = input_tensor[test_scenario_index]
            single_target_original = target_tensor[test_scenario_index]
            print(f"Forma de la entrada original del escenario: {single_input_original.shape}")
        except IndexError:
            print(f"Error fatal: No se pudo acceder al índice {test_scenario_index} en los tensores originales.")
            test_scenario_index = -1 # Marcar como inválido

    if test_scenario_index != -1:
        # === PASO 3: Realiza la predicción ===
        try:
            start_pred_time = time.time()
            prediction_denormalized = predict_single_scenario(
                final_model, single_input_original, scaler_input, scaler_target, DEVICE
            )
            end_pred_time = time.time()
            print(f"Predicción realizada en {end_pred_time - start_pred_time:.4f} segundos.")
            print(f"\nPredicción para el escenario {test_scenario_index} (Desnormalizada):")
            print("Forma:", prediction_denormalized.shape)
            print("Primeros 5 pasos:\n", prediction_denormalized[:5, :])

            # === PASO 4: (Opcional) Compara con el valor real ===
            if isinstance(single_target_original, torch.Tensor):
                target_np_original = single_target_original.cpu().numpy()
            else:
                 target_np_original = np.array(single_target_original)

            print(f"\nValor Real para el escenario {test_scenario_index} (Original):")
            print("Forma:", target_np_original.shape)
            print("Primeros 5 pasos:\n", target_np_original[:5, :])

            # Calcular error
            mae_escenario = np.mean(np.abs(prediction_denormalized - target_np_original))
            rmse_escenario = np.sqrt(np.mean((prediction_denormalized - target_np_original)**2))
            print(f"\nError Absoluto Medio (MAE) para este escenario: {mae_escenario:.5f}")
            print(f"Raíz del Error Cuadrático Medio (RMSE) para este escenario: {rmse_escenario:.5f}")

            # Verificar no negatividad final
            if np.any(prediction_denormalized < 0):
                 print("\n¡ADVERTENCIA! Se encontraron valores < 0 en la predicción desnormalizada final.")
            else:
                 print("\nConfirmado: No hay valores < 0 en la predicción desnormalizada final.")


        except Exception as e:
            print(f"\n*** Error durante la predicción del escenario específico: ***")
            print(e)
            traceback.print_exc()

else:
    print("\nNo se puede ejecutar la prueba de escenario específico.")
    print("Asegúrate de que el entrenamiento se completó y las variables")
    print(f"{required_vars_test} están disponibles en el entorno.")

print("=" * 60 + "\n")

Cargando y preprocesando datos...
Archivos .pt no encontrados, generando datos de ejemplo...
Datos de ejemplo generados.
Forma Input Tensor Original: torch.Size([500, 74, 6])
Forma Target Tensor Original: torch.Size([500, 74, 4])
Normalizando datos...
Datos normalizados correctamente.
Dividiendo datos...

--- DETALLES DE LA DIVISIÓN DE DATOS ---
Número total de escenarios originales: 500
Porcentaje para validación (test_size): 20.0%
Semilla aleatoria (random_state): 42
------------------------------------------------------------
Número de escenarios para ENTRENAMIENTO: 400
Índices originales usados para ENTRENAMIENTO (primeros 50):
[1, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 16, 17, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61]
...
------------------------------------------------------------
Número de escenarios para VALIDACIÓN: 100
Índices originales usados para VALIDACIÓN (primeros 50):
[0

[I 2025-04-16 02:37:06,102] A new study created in RDB with name: transformer_optimization_v1


Estudio Optuna 'transformer_optimization_v1' cargado/creado desde 'sqlite:///transformer_optimization_v1.db'.
Número de trials ya completados: 0

--- Optuna Trial 0 ---
Params: d_model=128, nhead=8, layers=5, ff=256(2x), drop=0.056, lr=0.000853, bs=16, wd=0.0000016


/opt/homebrew/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


    Epoch 01/30, Train Loss: 1.00477, Val Loss: 1.02005
    Epoch 02/30, Train Loss: 0.99794, Val Loss: 1.02005
    Epoch 03/30, Train Loss: 0.99794, Val Loss: 1.02005
    Epoch 04/30, Train Loss: 0.99794, Val Loss: 1.02005
    Epoch 05/30, Train Loss: 0.99794, Val Loss: 1.02005
    Epoch 06/30, Train Loss: 0.99794, Val Loss: 1.02005
    Epoch 07/30, Train Loss: 0.99794, Val Loss: 1.02005
    Epoch 08/30, Train Loss: 0.99794, Val Loss: 1.02005
    --> Optuna trial early stopping at epoch 8. Val Loss: 1.02005


...

    Epoch 01/30, Train Loss: 1.44805, Val Loss: 1.46518
    Epoch 02/30, Train Loss: 1.38513, Val Loss: 1.36717
    Epoch 03/30, Train Loss: 1.30759, Val Loss: 1.28447
    Epoch 04/30, Train Loss: 1.26414, Val Loss: 1.21560
    Epoch 05/30, Train Loss: 1.22201, Val Loss: 1.15942


[I 2025-04-16 02:46:23,838] Trial 24 pruned. 


    Epoch 06/30, Train Loss: 1.16946, Val Loss: 1.11433
    --> Trial 24 podado en la época 6.

--- OPTIMIZACIÓN FINALIZADA ---
Mejor Valor (Val Loss): 1.00821
Mejores Hiperparámetros encontrados:
{'d_model': 256, 'ff_multiplier': 2, 'nhead': 16, 'num_encoder_layers': 3, 'dropout': 0.29867226031868743, 'learning_rate': 1.7633861657015756e-05, 'batch_size': 128, 'weight_decay': 2.9841239840264177e-07}

--- INICIANDO ENTRENAMIENTO FINAL ---
Usando parámetros: {'d_model': 256, 'ff_multiplier': 2, 'nhead': 16, 'num_encoder_layers': 3, 'dropout': 0.29867226031868743, 'learning_rate': 1.7633861657015756e-05, 'batch_size': 128, 'weight_decay': 2.9841239840264177e-07, 'calculated_dim_feedforward': 512}


Python(78943) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78944) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78977) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78978) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Epoch 001/150, Train Loss: 1.43308, Val Loss: 1.27109, LR: 0.000018, Time: 5.20s
    --> Modelo guardado en best_transformer_model_relu.pth (Val Loss: 1.27109)
Epoch 002/150, Train Loss: 1.28258, Val Loss: 1.11978, LR: 0.000018, Time: 1.55s
    --> Modelo guardado en best_transformer_model_relu.pth (Val Loss: 1.11978)
Epoch 003/150, Train Loss: 1.16825, Val Loss: 1.04327, LR: 0.000018, Time: 1.45s
    --> Modelo guardado en best_transformer_model_relu.pth (Val Loss: 1.04327)
Epoch 004/150, Train Loss: 1.08491, Val Loss: 1.01537, LR: 0.000018, Time: 1.32s
    --> Modelo guardado en best_transformer_model_relu.pth (Val Loss: 1.01537)
Epoch 005/150, Train Loss: 1.05102, Val Loss: 1.00914, LR: 0.000018, Time: 1.27s
    --> Modelo guardado en best_transformer_model_relu.pth (Val Loss: 1.00914)
Epoch 006/150, Train Loss: 1.01522, Val Loss: 1.00835, LR: 0.000018, Time: 1.09s
    --> Modelo guardado en best_transformer_model_relu.pth (Val Loss: 1.00835)
Epoch 007/150, Train Loss: 1.01349, Val 

*******

# Validación del modelo

In [21]:
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import math

# Definir la clase PositionalEncoding (necesaria para el Transformer)
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

# Definir la clase del modelo Transformer (necesaria para cargar el modelo)
class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, d_model, nhead, num_encoder_layers, dim_feedforward, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.d_model = d_model

        # Proyección lineal para ajustar la dimensión de entrada a d_model
        self.input_proj = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)

        # Capas del Transformer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)

        # Proyección lineal para la salida
        self.output_proj = nn.Linear(d_model, output_dim)

    def forward(self, src):
        # src shape: (batch_size, seq_len, input_dim)
        src = self.input_proj(src)  # Proyectar a d_model
        src = src.permute(1, 0, 2)  # Cambiar a (seq_len, batch_size, d_model) para Transformer
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = output.permute(1, 0, 2)  # Volver a (batch_size, seq_len, d_model)
        output = self.output_proj(output)  # Proyectar a output_dim
        return output

# Cargar el modelo guardado
def load_model(model_path, input_dim, output_dim, d_model, nhead, num_encoder_layers, dim_feedforward, dropout, device):
    model = TransformerModel(
        input_dim=input_dim,
        output_dim=output_dim,
        d_model=d_model,
        nhead=nhead,
        num_encoder_layers=num_encoder_layers,
        dim_feedforward=dim_feedforward,
        dropout=dropout
    ).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Función para evaluar el modelo y calcular métricas
def evaluate_model(model, data_loader, criterion, scaler_target, device):
    model.eval()
    all_preds = []
    all_targets = []
    total_loss = 0.0

    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input, batch_target = batch_input.to(device), batch_target.to(device)
            output = model(batch_input)
            loss = criterion(output, batch_target)
            total_loss += loss.item()

            # Guardar predicciones y valores reales
            all_preds.append(output.cpu().numpy())
            all_targets.append(batch_target.cpu().numpy())

    # Concatenar todas las predicciones y valores reales
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)

    # Desnormalizar las predicciones y los valores reales
    all_preds_denorm = scaler_target.inverse_transform(all_preds.reshape(-1, all_preds.shape[-1])).reshape(all_preds.shape)
    all_targets_denorm = scaler_target.inverse_transform(all_targets.reshape(-1, all_targets.shape[-1])).reshape(all_targets.shape)

    # Calcular métricas en la escala original
    mse = mean_squared_error(all_targets_denorm.reshape(-1), all_preds_denorm.reshape(-1))
    mae = mean_absolute_error(all_targets_denorm.reshape(-1), all_preds_denorm.reshape(-1))
    r2 = r2_score(all_targets_denorm.reshape(-1), all_preds_denorm.reshape(-1))

    avg_loss = total_loss / len(data_loader)
    return avg_loss, mse, mae, r2, all_preds_denorm, all_targets_denorm

# Función para probar el modelo con un ejemplo específico
def test_model(model, input_data, scaler_input, scaler_target, device):
    model.eval()
    with torch.no_grad():
        # Normalizar el dato de entrada
        input_np = input_data.numpy().reshape(-1, input_data.shape[-1])
        input_normalized = scaler_input.transform(input_np).reshape(input_data.shape)
        input_tensor = torch.tensor(input_normalized, dtype=torch.float32).to(device)

        # Hacer la predicción
        if len(input_tensor.shape) == 2:  # Si es un solo ejemplo, agregar dimensión de batch
            input_tensor = input_tensor.unsqueeze(0)
        output = model(input_tensor)

        # Desnormalizar la predicción
        output_np = output.cpu().numpy()
        output_denorm = scaler_target.inverse_transform(output_np.reshape(-1, output_np.shape[-1])).reshape(output_np.shape)
        return output_denorm

# Configuración
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hiperparámetros del modelo (AJUSTA ESTOS VALORES SEGÚN LOS MEJORES HIPERPARÁMETROS DE OPTUNA)
# Reemplaza estos valores con los que obtuviste al entrenar el modelo
# Hiperparámetros del modelo (AJUSTA ESTOS VALORES SEGÚN LOS DEL MODELO GUARDADO)
# Reemplaza estos valores con los que obtuviste al entrenar el modelo
d_model = 256              # <--- Cambiado de 128
nhead = 4                  # Asumiendo que este no cambió, pero verifica si es necesario
num_encoder_layers = 2     # <--- Cambiado de 3
dim_feedforward = 256      # <--- Cambiado de 512
dropout = 0.1              # Asumiendo que este no cambió, pero verifica si es necesario
input_dim = 6              # Probablemente correcto
output_dim = 4             # Probablemente correcto
batch_size = 32            # Este afecta al DataLoader, no a la estructura del modelo

# Cargar el modelo
model_path = 'best_transformer_model.pth'
model = load_model(model_path, input_dim, output_dim, d_model, nhead, num_encoder_layers, dim_feedforward, dropout, device)

# Crear DataLoader para los datos de validación (o prueba)
# Asume que tienes val_input y val_target del código anterior
val_dataset = TensorDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Definir el criterio de pérdida
criterion = nn.MSELoss()

# Evaluar el modelo en los datos de validación
avg_loss, mse, mae, r2, predictions, true_values = evaluate_model(model, val_loader, criterion, scaler_target, device)

# Mostrar estadísticas
print("Estadísticas del modelo en los datos de validación:")
print(f"Pérdida promedio (MSE normalizado): {avg_loss:.4f}")
print(f"Error Cuadrático Medio (MSE desnormalizado): {mse:.4f}")
print(f"Error Absoluto Medio (MAE desnormalizado): {mae:.4f}")
print(f"Coeficiente de Determinación (R² desnormalizado): {r2:.4f}")

# Probar el modelo con un ejemplo específico (por ejemplo, el primer escenario de validación)
test_input = val_input[0]  # Primer escenario de validación
predicted_output = test_model(model, test_input, scaler_input, scaler_target, device)
true_output = scaler_target.inverse_transform(val_target[0].numpy().reshape(-1, val_target.shape[-1])).reshape(val_target[0].shape)

# Mostrar los resultados de la prueba
print("\nPrueba con un ejemplo específico (primer escenario de validación):")
print("Predicciones (desnormalizadas) para los primeros 5 pares de relés:")
for i in range(min(74, predicted_output.shape[1])):  # Mostrar solo los primeros 5 pares
    print(f"Par {i+1}:")
    print(f"  Predicción: {predicted_output[0, i, :]}")
    print(f"  Valor real: {true_output[i, :]}")

/opt/homebrew/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Estadísticas del modelo en los datos de validación:
Pérdida promedio (MSE normalizado): 1.8232
Error Cuadrático Medio (MSE desnormalizado): 16.5326
Error Absoluto Medio (MAE desnormalizado): 3.1601
Coeficiente de Determinación (R² desnormalizado): -0.8002

Prueba con un ejemplo específico (primer escenario de validación):
Predicciones (desnormalizadas) para los primeros 5 pares de relés:
Par 1:
  Predicción: [ 0.43002382  7.486592   12.017996    4.080612  ]
  Valor real: [10.739031   1.9509555  5.9624233  1.2299176]
Par 2:
  Predicción: [0.11465684 5.9169545  9.741835   2.696026  ]
  Valor real: [1.0115863 4.9236174 3.1815815 7.994131 ]
Par 3:
  Predicción: [ 1.259947    5.960698    2.5113537  -0.33478466]
  Valor real: [9.688989  9.712443  3.1813238 5.280407 ]
Par 4:
  Predicción: [0.7969634 1.1996793 9.026089  2.500911 ]
  Valor real: [ 6.613281   3.8063807  1.4643133 17.902504 ]
Par 5:
  Predicción: [2.8568864 2.7802656 6.520256  5.565122 ]
  Valor real: [2.7434456 2.0836818 3.99795